# Testing Torch Utils

In [1]:
import sys

In [2]:
project_folder = "../"
sys.path.insert(0, project_folder)
print(sys.path) # View the path and verify

['../', '/Users/federicosiciliano/Desktop/Py_utils/easy_torch/ntb', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python38.zip', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/lib-dynload', '', '/Users/federicosiciliano/Library/Python/3.8/lib/python/site-packages', '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/site-packages']


In [3]:
import easy_data, easy_exp, easy_torch

## Configuration loading

In [4]:
cfg = easy_exp.cfg.load_configuration("config_nn")

In [5]:
cfg

{'__exp__': {'name': 'prova',
  'project_folder': '../',
  'key_len': 16,
  'key_prefix': '',
  '__imports__': ['torchvision'],
  '__nosave__': {'model.loader_params.num_workers': None,
   'model.trainer_params.accelerator': None,
   'model.trainer_params.enable_checkpointing': None,
   'model.trainer_params.logger': None,
   'model.trainer_params.callbacks.1.ModelCheckpoint.dirpath': None,
   'model.trainer_params.callbacks.1.ModelCheckpoint.filename': None}},
 'data': {'name': 'MNIST',
  'source': 'torchvision',
  'merge_before_split': False,
  'split_keys': {'train_x': ['train_x', 'val_x'],
   'train_y': ['train_y', 'val_y']},
  'train_sizes': [100, 100],
  'test_sizes': [0.2],
  'split_random_state': 21094,
  'one_hot_encode': True,
  'scaling': 'MinMax'},
 'model': {'name': 'resnet18',
  'torchvision_params': {'weights': None},
  'optimizer': {'name': 'Adam', 'params': {'lr': 0.1, 'weight_decay': 0.0005}},
  'loss': 'CrossEntropyLoss',
  'loader_params': {'batch_size': 256, 'num_w

In [6]:
data = easy_data.data.load_data(**cfg["data"])

In [7]:
loaders = easy_torch.preparation.prepare_data_loaders(data, cfg["model"]["loader_params"])

In [8]:
cfg["model"]["in_channels"] = data["train_x"].shape[1]
cfg["model"]["out_features"] = data["train_y"].shape[1]

In [9]:
main_module = easy_torch.model.get_torchvision_model(**cfg["model"])

In [10]:
exp_found, experiment_id = easy_exp.exp.get_experiment_id(cfg)

In [11]:
print(exp_found, experiment_id)

False n6ccQw44ZpQGH31h


In [13]:
# Set experiment_id in trainer_params
trainer_params = easy_torch.preparation.prepare_experiment_id(cfg["model"]["trainer_params"], experiment_id)

In [14]:
trainer_params["callbacks"] = easy_torch.preparation.prepare_callbacks(trainer_params)
trainer_params["logger"] = easy_torch.preparation.prepare_logger(trainer_params)
trainer = easy_torch.preparation.prepare_trainer(**trainer_params)

# callbacks = easy_torch.preparation.prepare_callbacks(cfg["model"]["trainer_params"])
# logger = easy_torch.preparation.prepare_logger(cfg["model"]["trainer_params"])
# already_defined = ["callbacks","logger"]
# trainer = easy_torch.preparation.prepare_trainer(**{k:cfg["model"]["trainer_params"][k] for k in cfg["model"]["trainer_params"] if k not in already_defined}, callbacks=callbacks, logger=logger)

Global seed set to 42
Global seed set to 42
Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [16]:
loss = easy_torch.preparation.prepare_loss(cfg["model"]["loss"])

In [17]:
optimizer = easy_torch.preparation.prepare_optimizer(**cfg["model"]["optimizer"])

In [18]:
model = easy_torch.process.create_model(main_module, loss, optimizer)

Global seed set to 42


In [20]:
easy_torch.process.train_model(trainer, model, loaders)


  | Name        | Type             | Params
-------------------------------------------------
0 | main_module | ResNet           | 11.2 M
1 | loss        | CrossEntropyLoss | 0     
-------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.701    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/federicosiciliano/Library/Python/3.8/lib/python/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/federicosiciliano/Library/Python/3.8/lib/python/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/federicosiciliano/Library/Python/3.8/lib/python/site-packages/pytorch_lightning/trainer/trainer.py:1933: PossibleUserWarning: The number of training batches 

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [21]:
easy_torch.process.test_model(trainer, model, loaders)

/Users/federicosiciliano/Library/Python/3.8/lib/python/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │      6171157594112.0      │
└───────────────────────────┴───────────────────────────┘

In [22]:
easy_exp.exp.save_experiment(cfg)